In [1]:
# Author: Anonymous
# License: CC-BY-NC-SA 4.0

In [2]:
import os
import pandas as pd
import geopandas as gpd
import networkx as nx
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx
import numpy as np

/opt/homebrew/Caskroom/miniconda/base/envs/mapillary/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
# --- CONFIGURATION ---
print("--- Initializing Unified Analysis Configuration ---")
PROJECT_ROOT = os.path.abspath('..') 
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results")
FIGURES_DIR = os.path.join(RESULTS_DIR, "figures")
os.makedirs(FIGURES_DIR, exist_ok=True)
print(f"Outputs will be saved to: {RESULTS_DIR} and {FIGURES_DIR}")

# --- INPUT FILES ---
INPUT_CLUSTERS_CSV = os.path.join(RESULTS_DIR, "full_network_with_clusters.csv")
TEMP_ADJACENCY_PKL = os.path.join(RESULTS_DIR, "temp_full_adjacency_list.pkl")

# --- OUTPUT FILES ---
OUTPUT_METRICS_CSV = os.path.join(RESULTS_DIR, "final_city_metrics.csv")

# --- PARAMETERS ---
# The core, justifiable decision of this analysis.
# We define which cluster IDs are considered "supportive".
SUPPORTIVE_CLUSTER_IDS = [0, 2] # Clusters A and B
CRS = "EPSG:3857"
# Cities for detailed maps, chosen to represent different quadrants in the new typology.
CITIES_TO_MAP = ["paris", "san_francisco", "boston", "hamburg", "coimbra", "fortaleza", "apia", "moscow", "kyiv", "doha", "mexico_city"]

--- Initializing Unified Analysis Configuration ---
Outputs will be saved to: /Users/Codes/results/figures


In [4]:
def build_adjacency_list(gdf: gpd.GeoDataFrame) -> dict:
    """
    Builds a dictionary mapping each segment's unique_edge_id to a list of
    its neighboring unique_edge_ids by checking for touching endpoints.
    This is the core of the geometric topology inference.
    """
    print("Building spatial index for all segment endpoints...")
    # Create a new GeoDataFrame containing every endpoint
    endpoints = []
    for idx, row in gdf.iterrows():
        # Add the start point
        endpoints.append({'unique_edge_id': row['unique_edge_id'], 'geometry': Point(row['geometry'].coords[0])})
        # Add the end point
        endpoints.append({'unique_edge_id': row['unique_edge_id'], 'geometry': Point(row['geometry'].coords[-1])})
    
    endpoints_gdf = gpd.GeoDataFrame(endpoints, crs=CRS)
    spatial_index = endpoints_gdf.sindex

    print("Inferring network adjacency from endpoint proximity...")
    adjacency = {uid: [] for uid in gdf['unique_edge_id']}
    
    # Iterate through each segment to find its neighbors
    for idx, segment in tqdm(gdf.iterrows(), total=len(gdf), desc="Finding Neighbors"):
        uid_self = segment['unique_edge_id']
        
        # Get start and end points of the current segment
        start_node = Point(segment['geometry'].coords[0])
        end_node = Point(segment['geometry'].coords[-1])
        
        # Find all endpoints that are very close to the start_node of our segment
        # We use a small buffer (1 meter) to account for tiny floating point inaccuracies
        possible_matches_at_start = spatial_index.query(start_node.buffer(1), predicate="intersects")
        
        for match_idx in possible_matches_at_start:
            uid_neighbor = endpoints_gdf.iloc[match_idx]['unique_edge_id']
            if uid_neighbor != uid_self: # Don't connect a segment to itself
                adjacency[uid_self].append(uid_neighbor)

        # Do the same for the end_node
        possible_matches_at_end = spatial_index.query(end_node.buffer(1), predicate="intersects")
        for match_idx in possible_matches_at_end:
            uid_neighbor = endpoints_gdf.iloc[match_idx]['unique_edge_id']
            if uid_neighbor != uid_self:
                adjacency[uid_self].append(uid_neighbor)

    # Clean up the adjacency list by removing duplicate neighbors
    for uid, neighbors in adjacency.items():
        adjacency[uid] = list(set(neighbors))
        
    return adjacency

In [5]:
def calculate_core_metrics(city_gdf: gpd.GeoDataFrame, city_adjacency: dict) -> tuple:
    """
    Calculates the two core metrics for a single city:
    1. Proportion of Supportive Roads (Quantity)
    2. CNRI (Connectivity/Cohesion)
    Returns: (psr, cnri, lcc_uids_for_plotting)
    """
    total_city_length = city_gdf['length'].sum()
    if total_city_length == 0:
        return 0.0, 0.0, set()

    supportive_segments_df = city_gdf[city_gdf['cluster'].isin(SUPPORTIVE_CLUSTER_IDS)]
    
    # --- METRIC 1: PSR (QUANTITY) ---
    total_supportive_length = supportive_segments_df['length'].sum()
    psr = total_supportive_length / total_city_length
    
    if supportive_segments_df.empty or total_supportive_length == 0:
        return psr, 0.0, set()

    # --- METRIC 2: CNRI (CONNECTIVITY) ---
    supportive_uids = set(supportive_segments_df['unique_edge_id'])
    
    G_supportive = nx.Graph()
    for uid_self in supportive_uids:
        if uid_self in city_adjacency:
            for uid_neighbor in city_adjacency[uid_self]:
                if uid_neighbor in supportive_uids:
                    G_supportive.add_edge(uid_self, uid_neighbor)

    if G_supportive.number_of_nodes() == 0:
        return psr, 0.0, set()

    components = list(nx.connected_components(G_supportive))
    if not components:
        return psr, 0.0, set()

    lcc_uids = max(components, key=len)
    lcc_length = supportive_segments_df[supportive_segments_df['unique_edge_id'].isin(lcc_uids)]['length'].sum()
    
    cnri = lcc_length / total_supportive_length
    
    return psr, cnri, lcc_uids

In [6]:
def plot_city_typology(summary_df: pd.DataFrame):
    """
    Generates the new, robust 2D scatter plot based on the two core metrics.
    """
    print("Generating City Typology scatter plot...")
    plt.style.use('seaborn-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 10))

    summary_df['Status'] = np.where(summary_df['city_id'].isin(CITIES_TO_MAP), 'Case Study City', 'Other City')
    color_palette = {'Case Study City': '#e41a1c', 'Other City': '#377eb8'}

    x_axis = 'PSR'
    y_axis = 'CNRI'

    sns.scatterplot(data=summary_df, x=x_axis, y=y_axis, ax=ax, s=100, alpha=0.9, legend=False,
                    edgecolor='black', hue='Status', palette=color_palette,
                    hue_order=['Other City', 'Case Study City'])

    median_x = summary_df[x_axis].median()
    median_y = summary_df[y_axis].median()
    ax.axvline(median_x, color='grey', linestyle='--', lw=1.5)
    ax.axhline(median_y, color='grey', linestyle='--', lw=1.5)

    # Quadrant Labels
    ax.text(ax.get_xlim()[1], ax.get_ylim()[1], ' Holistically Ready ', ha='right', va='top', fontsize=14, fontweight='bold', alpha=0.7)
    ax.text(ax.get_xlim()[0], ax.get_ylim()[1], ' Efficient Core ', ha='left', va='top', fontsize=14, fontweight='bold', alpha=0.7)
    ax.text(ax.get_xlim()[1], ax.get_ylim()[0], ' Fragmented Excellence ', ha='right', va='bottom', fontsize=14, fontweight='bold', alpha=0.7)
    ax.text(ax.get_xlim()[0], ax.get_ylim()[0], ' Fundamentally Unready ', ha='left', va='bottom', fontsize=14, fontweight='bold', alpha=0.7)
    
    # Add text labels for the highlighted cities
    for city_id in CITIES_TO_MAP:
        city_data = summary_df[summary_df['city_id'] == city_id]
        if not city_data.empty:
            ax.text(city_data[x_axis].iloc[0], city_data[y_axis].iloc[0] + 0.015, city_id.replace('_', ' ').title(),
                    fontsize=11, ha='center', fontweight='bold')

    # Titles and labels
    ax.set_title('City Typology: Quantity vs. Connectivity of Supportive Infrastructure', fontsize=20, pad=20)
    ax.set_xlabel('PSR (by length)', fontsize=14)
    ax.set_ylabel('CNRI (Connectivity of Supportive Roads)', fontsize=14)
    
    plt.tight_layout()
    output_path = os.path.join(FIGURES_DIR, "city_readiness_quadrant_plot.png")
    plt.savefig(output_path, dpi=300)
    print(f"Definitive typology plot saved to: {output_path}")
    plt.close()

In [7]:
def plot_cnri_map(city_gdf: gpd.GeoDataFrame, city_id: str, cnri: float, lcc_uids: set):
    """
    Generates a new, more informative side-by-side map that directly visualizes the CNRI.
    """
    print(f"Generating CNRI map for '{city_id}'...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    supportive_network = city_gdf[city_gdf['cluster'].isin(SUPPORTIVE_CLUSTER_IDS)]
    
    # --- Panel A: All Supportive Infrastructure (The Denominator) ---
    city_gdf.plot(ax=ax1, color='lightgray', linewidth=0.5)
    if not supportive_network.empty:
        supportive_network.plot(ax=ax1, color='cornflowerblue', linewidth=1.5)
    cx.add_basemap(ax1, crs=city_gdf.crs.to_string(), source=cx.providers.CartoDB.Positron)
    ax1.set_title(f"A: All Supportive Infrastructure (Clusters A & B)", fontsize=16)
    ax1.set_axis_off()

    # --- Panel B: The Largest Connected Component (The Numerator) ---
    lcc_network = supportive_network[supportive_network['unique_edge_id'].isin(lcc_uids)]
    
    city_gdf.plot(ax=ax2, color='lightgray', linewidth=0.5)
    if not supportive_network.empty:
        # Plot all supportive segments faintly in the background for context
        supportive_network.plot(ax=ax2, color='lightcoral', linewidth=0.7)
    if not lcc_network.empty:
        # Plot the LCC boldly on top
        lcc_network.plot(ax=ax2, color='red', linewidth=2.0)
    cx.add_basemap(ax2, crs=city_gdf.crs.to_string(), source=cx.providers.CartoDB.Positron)
    ax2.set_title(f"B: Largest Connected Component (LCC)", fontsize=16)
    ax2.set_axis_off()

    fig.suptitle(f'Core Network Cohesion: {city_id.replace("_", " ").title()} (CNRI = {cnri:.3f})', fontsize=24)
    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
    
    output_path = os.path.join(FIGURES_DIR, f"map_comparison_{city_id}.png")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Comparison map for '{city_id}' saved to: {output_path}")
    plt.close()

In [8]:
def main():
    """Main function to orchestrate the robust analysis and visualization workflow."""
    print("--- Starting Unified Robust Analysis ---")

    # 1. Load data with cluster assignments
    print(f"Loading data from: {INPUT_CLUSTERS_CSV}...")
    df = pd.read_csv(INPUT_CLUSTERS_CSV, sep=";", low_memory=False)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=CRS)
    gdf['length'] = pd.to_numeric(gdf['length'], errors='coerce').fillna(0)
    print("Data loaded successfully.")

    # 2. Load or build the full adjacency list
    if os.path.exists(TEMP_ADJACENCY_PKL):
        print(f"Loading pre-computed adjacency list from: {TEMP_ADJACENCY_PKL}")
        full_adjacency_list = joblib.load(TEMP_ADJACENCY_PKL)
    else:
        print("Adjacency list not found. Building it now...")
        full_adjacency_list = build_adjacency_list(gdf)
        joblib.dump(full_adjacency_list, TEMP_ADJACENCY_PKL)
    print("Adjacency list is ready.")
    
    # 3. Calculate Core Metrics for each city
    print("\nCalculating Core Metrics (PSR & CNRI) for all 100 cities...")
    all_results = []
    city_lcc_data = {} # To store LCC UIDs for plotting

    for city_id, group in tqdm(gdf.groupby('city_id'), desc="Processing Cities"):
        city_uids = set(group['unique_edge_id'])
        city_adjacency = {uid: [neighbor for neighbor in full_adjacency_list.get(uid, []) if neighbor in city_uids] 
                          for uid in city_uids}
        
        proportion, cnri, lcc_uids = calculate_core_metrics(group.copy(), city_adjacency)
        
        all_results.append({
            'city_id': city_id, 
            'PSR': proportion,
            'CNRI': cnri
        })
        city_lcc_data[city_id] = lcc_uids

    summary_df = pd.DataFrame(all_results).sort_values('CNRI', ascending=False)
    
    # 4. Save the final, robust metrics
    summary_df.to_csv(OUTPUT_METRICS_CSV, index=False)
    print(f"\n✅ Final city metrics saved to: {OUTPUT_METRICS_CSV}")
    print("\n--- Top 10 Cities by CNRI ---")
    print(summary_df.head(10).to_string())

    # 5. Generate all figures
    print("\n--- Generating All Figures ---")
    plot_city_typology(summary_df)
    
    print("\n--- Generating Detailed City Maps ---")
    for city_id in CITIES_TO_MAP:
        city_gdf = gdf[gdf['city_id'] == city_id]
        city_metrics = summary_df[summary_df['city_id'] == city_id].iloc[0]
        plot_cnri_map(city_gdf, city_id, city_metrics['CNRI'], city_lcc_data[city_id])

    print("\n--- Unified script complete. ---")

In [9]:
if __name__ == "__main__":
    main()

--- Starting Unified Robust Analysis ---
Loading data from: /Users/results/full_network_with_clusters.csv...
Data loaded successfully.
Loading pre-computed adjacency list from: /Users/results/temp_full_adjacency_list.pkl
Adjacency list is ready.

Calculating Core Metrics (Proportion & Core NRI) for all 100 cities...


Processing Cities: 100%|██████████████████████| 100/100 [00:17<00:00,  5.73it/s]



✅ Final city metrics saved to: /Users/results/final_city_metrics.csv

--- Top 10 Cities by CNRI ---
                   city_id               PSR       CNRI
5                     apia               0.043253  0.957332
66                  moscow               0.651635  0.937737
51                    kyiv               0.556159  0.897225
36                    doha               0.174281  0.817649
61             mexico_city               0.144383  0.684430
84                santiago               0.255517  0.645172
20            buenos_aires               0.165147  0.622258
68  municipio_de_sao_paulo               0.137207  0.586303
79               reykjavik               0.044946  0.574322
44                 houston               0.165697  0.540729

--- Generating All Figures ---
Generating City Typology scatter plot...
Definitive typology plot saved to: /Users/results/figures/city_readiness_quadrant_plot.png

--- Generating Detailed City Maps ---
Generating CNRI map for 'paris'...
Compa